## Micropython memory map visualizer

The below class is used to visualize the memory map of MicroPython. It is based on the `micropython.memory_info(1)` output.  
It can be used to visualize the current usage of the memory befor and after gc collection, or to visualize the difference between two memory maps.

*Note:* Currently the Memoryinfo class is included in this notebook, but the intent is for it to be moved into the magic module in the future.


In [1]:
from micropython_magic.memoryinfo import MemoryInfo

In [2]:
# issue soft reset before running
%mpy --soft-reset
mem_lst = %mpy import micropython; micropython.mem_info(True)
mem_lst

['stack: 516 out of 7936', 'GC: total: 193664, used: 4496, free: 189168', ' No. of 1-blocks: 53, 2-blocks: 14, max blk sz: 64, max free sz: 11813', 'GC memory layout; from 20010b80:', '00000000: h=MhhhhBD.hBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh=', '00000400: ==BB=h===========TB=h=h===================h=====h==h============', '00000800: =====h==========================================================', '00000c00: =====h==========================================================', '00001000: =====h=hhhBhhhhBh=h..h..h=.....h=h=.............................', '       (184 lines all free)', '0002f400: ........']

In [3]:
# create the memory map using the output, and display it
# map_1 = MemoryInfo(mem_lst.data, columns=1)
map_1 = MemoryInfo(mem_lst.data, cols=1)
map_1

Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_1190 of Total: 0x002_f480  free: 0x002_e2f0 pct free: 97.7%
1-Blocks:          53  2-Blocks:         14
Max Block size:    64  Max Free size: 11813
h=MhhhhBD.hBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= 
==BB=h===========TB=h=h===================h=====h==h============ 
=====h========================================================== 
=====h========================================================== 
=====h=hhhBhhhhBh=h..h..h=.....h=h=............................. 
........

In [4]:
# default = 4 columns
map_1.columns = 2
map_1

Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_1190 of Total: 0x002_f480  free: 0x002_e2f0 pct free: 97.7%
1-Blocks:          53  2-Blocks:         14
Max Block size:    64  Max Free size: 11813
h=MhhhhBD.hBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= ==BB=h===========TB=h=h===================h=====h==h============ 
=====h========================================================== =====h========================================================== 
=====h=hhhBhhhhBh=h..h..h=.....h=h=............................. ........

In [5]:
map_1.rainbow = True  # alternating colors for each block
map_1

Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_1190 of Total: 0x002_f480  free: 0x002_e2f0 pct free: 97.7%
1-Blocks:          53  2-Blocks:         14
Max Block size:    64  Max Free size: 11813
h=MhhhhBD.hBTTBDhTBDBBBh===DBDh====B=BBBBBBTB=BTB=BBBTB=TBTB=Bh= ==BB=h===========TB=h=h===================h=====h==h============ 
=====h========================================================== =====h========================================================== 
=====h=hhhBhhhhBh=h..h..h=.....h=h=............................. ........

In [6]:
# consume some more memory
%mpy print(help('modules'))
%mpy import gc; gc.collect()

mem_lst = %mpy import micropython; micropython.mem_info(True)
map_2=MemoryInfo(mem_lst.data, cols=2 )
map_2.rainbow = True
map_2

Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_1080 of Total: 0x002_f480  free: 0x002_e400 pct free: 97.8%
1-Blocks:          36  2-Blocks:         14
Max Block size:    64  Max Free size: 11747
h=Mhhhh=DhhhhhBDh..DBBBh===DBDh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh= ==BB=h===========.B=h=h===================h=====h==h============ 
=====h========================================================== =====h========================================================== 
=====........h=.h=..h=.......................................... ................hh.B...............h=........................... 
........

In [7]:
# %%micropython
# eat even more memory - with disabled gc
import gc

gc.disable()
foo = "Parrot"
try:
    for _ in range(15):
        foo = foo + foo
except MemoryError as e:
    print(e)
    pass

['memory allocation failed, allocating 98305 bytes']

In [8]:
# take anoter memory snapshot 
mem_lst = %mpy micropython.mem_info(True)
map_3=MemoryInfo(mem_lst.data)
map_3

Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x001_a2a0 of Total: 0x002_f480  free: 0x001_51e0 pct free: 44.7%
1-Blocks:          89  2-Blocks:         21
Max Block size:  3073  Max Free size:  5397
h=Mhhhh=DhhhhhBDhhhDBBBh===DBDh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh= ==BB=h===========SB=h=h===================h=====hhhh============ =====h========================================================== =====h========================================================== 
=====hSh=Bhhhh=hh=h=h=BhhhhBShh===h=h=hBhhShSSSh=h=SSSh===h===h= ======h========ShhSBh=============Sh=h========================== =============================================h================== =====================================================h========== 
===hhhhhh====h======SSh====h=====h============h================= =======h================================================h======= ================================================================ =========================h====================================

In [9]:
map_3.rainbow = True
map_3

Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x001_a2a0 of Total: 0x002_f480  free: 0x001_51e0 pct free: 44.7%
1-Blocks:          89  2-Blocks:         21
Max Block size:  3073  Max Free size:  5397
h=Mhhhh=DhhhhhBDhhhDBBBh===DBDh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh= ==BB=h===========SB=h=h===================h=====hhhh============ =====h========================================================== =====h========================================================== 
=====hSh=Bhhhh=hh=h=h=BhhhhBShh===h=h=hBhhShSSSh=h=SSSh===h===h= ======h========ShhSBh=============Sh=h========================== =============================================h================== =====================================================h========== 
===hhhhhh====h======SSh====h=====h============h================= =======h================================================h======= ================================================================ =========================h====================================

In [10]:
# compare snapshot 1 and 2 to see
#  - what remained the same = Black on Blue
#  - what was freed up = Green on Green ( can also be due to additional pages being reported)
#  - what was allocated = Yellow or White on Red

map_3 - map_1

 --> 
Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x001_a2a0 of Total: 0x002_f480  free: 0x001_51e0 pct free: 44.7%
1-Blocks:          89  2-Blocks:         21
Max Block size:  3073  Max Free size:  5397
h=Mhhhh=DhhhhhBDhhhDBBBh===DBDh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh= ==BB=h===========SB=h=h===================h=====hhhh============ =====h========================================================== =====h========================================================== 
=====hSh=Bhhhh=hh=h=h=BhhhhBShh===h=h=hBhhShSSSh=h=SSSh===h===h= ======h========ShhSBh=============Sh=h========================== =============================================h================== =====================================================h========== 
===hhhhhh====h======SSh====h=====h============h================= =======h================================================h======= ================================================================ =========================h==============================

In [11]:
# now free up some memory and take another memory snapshot 
%mpy import gc; gc.collect()
mem_lst = %mpy import micropython; micropython.mem_info(True)
map_4=MemoryInfo(mem_lst.data )
map_4.rainbow = True
map_4.show_free = True
map_4

Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_d660 of Total: 0x002_f480  free: 0x002_1e20 pct free: 71.7%
1-Blocks:          38  2-Blocks:         14
Max Block size:  3073  Max Free size:  5385
h=Mhhhh=DhhhhhBDh..DBBBh===DBDh====B=BBBBBB.B=B.B=BBB.B=hB.B=Bh= ==BB=h===========.B=h=h===================h=====...h============ =====h========================================================== =====h========================================================== 
=====.....h=..h=h=............................................h= ======h========......................h========================== =============================================................... ................................................................ 
........h====........S.......................................... .............................h================================== ================================================================ ==============================================================

In [12]:

# and show the difference between the last two snapshots
map_4-map_3

 --> 
Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_d660 of Total: 0x002_f480  free: 0x002_1e20 pct free: 71.7%
1-Blocks:          38  2-Blocks:         14
Max Block size:  3073  Max Free size:  5385
h=Mhhhh=DhhhhhBDh..DBBBh===DBDh====B=BBBBBB.B=B.B=BBB.B=hB.B=Bh= ==BB=h===========.B=h=h===================h=====...h============ =====h========================================================== =====h========================================================== 
=====.....h=..h=h=............................................h= ======h========......................h========================== =============================================................... ................................................................ 
........h====........S.......................................... .............................h================================== ================================================================ ========================================================

In [13]:
# or the reverse to see what got freed ( on RED)
map_3 - map_4

 --> 
Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x001_a2a0 of Total: 0x002_f480  free: 0x001_51e0 pct free: 44.7%
1-Blocks:          89  2-Blocks:         21
Max Block size:  3073  Max Free size:  5397
h=Mhhhh=DhhhhhBDhhhDBBBh===DBDh====B=BBBBBBhB=BhB=BBBBB=hBhB=Bh= ==BB=h===========SB=h=h===================h=====hhhh============ =====h========================================================== =====h========================================================== 
=====hSh=Bhhhh=hh=h=h=BhhhhBShh===h=h=hBhhShSSSh=h=SSSh===h===h= ======h========ShhSBh=============Sh=h========================== =============================================h================== =====================================================h========== 
===hhhhhh====h======SSh====h=====h============h================= =======h================================================h======= ================================================================ =========================h==============================

In [14]:
# are we back to where we started?
map_4 - map_1

 --> 
Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_d660 of Total: 0x002_f480  free: 0x002_1e20 pct free: 71.7%
1-Blocks:          38  2-Blocks:         14
Max Block size:  3073  Max Free size:  5385
h=Mhhhh=DhhhhhBDh..DBBBh===DBDh====B=BBBBBB.B=B.B=BBB.B=hB.B=Bh= ==BB=h===========.B=h=h===================h=====...h============ =====h========================================================== =====h========================================================== 
=====.....h=..h=h=............................................h= ======h========......................h========================== =============================================................... ................................................................ 
........h====........S.......................................... .............................h================================== ================================================================ ========================================================

In [15]:
%mpy print(dir())

["['w', 'micropython', '_', 'foo', 'f', 'machine', 'gc', '__name__', 'rp2']"]

In [16]:
%mpy del foo, f, w
%mpy import gc; gc.collect()

# take another memory snapshot 
mem_lst = %mpy import micropython; micropython.mem_info(True)
map_5=MemoryInfo(mem_lst.data )

map_5-map_1

 --> 
Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_11b0 of Total: 0x002_f480  free: 0x002_e2d0 pct free: 97.7%
1-Blocks:          36  2-Blocks:         14
Max Block size:    64  Max Free size: 11579
h=Mhhhh=DhhhhhBDh..DBBBh===DBDh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh= ==BB=h===========.B=h=h===================h=====...h============ =====h========================================================== =====h========================================================== 
=====.....h=..h=h=.hh.B............................h=.........h= ======h========................................................. ........h====................................................... ........

In [17]:
map_5 - map_4

 --> 
Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_11b0 of Total: 0x002_f480  free: 0x002_e2d0 pct free: 97.7%
1-Blocks:          36  2-Blocks:         14
Max Block size:    64  Max Free size: 11579
h=Mhhhh=DhhhhhBDh..DBBBh===DBDh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh= ==BB=h===========.B=h=h===================h=====...h============ =====h========================================================== =====h========================================================== 
=====.....h=..h=h=.hh.B............................h=.........h= ======h========................................................. ........h====................................................... ........

In [18]:
map_5

 --> 
Stack used:  0x0204 of Total: 0x1f00  pct free: 93.5%
Memory used: 0x000_11b0 of Total: 0x002_f480  free: 0x002_e2d0 pct free: 97.7%
1-Blocks:          36  2-Blocks:         14
Max Block size:    64  Max Free size: 11579
h=Mhhhh=DhhhhhBDh..DBBBh===DBDh====B=BBBBBB.B=B.B=BBB.B=.B.B=Bh= ==BB=h===========.B=h=h===================h=====...h============ =====h========================================================== =====h========================================================== 
=====.....h=..h=h=.hh.B............................h=.........h= ======h========................................................. ........h====................................................... ........